In [1]:
import flair.datasets
from pathlib import Path
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import TokenEmbeddings, FlairEmbeddings, WordEmbeddings, PooledFlairEmbeddings, StackedEmbeddings, CharacterEmbeddings, CharLMEmbeddings
from flair.models import LanguageModel
from gensim.models.keyedvectors import KeyedVectors
from typing import List
import os

In [2]:
# Disease
# NCBI_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/NCBI-disease-IOB'
#BioNLP13PC_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BioNLP13PC-IOB'
# BC5CDR_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC5CDR-disease-IOB'
# Genes
BC2GM_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC2GM-IOB'
JNLPBA_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/JNLPBA'



# define columns
columns = {0: 'text', 1: 'ner'}

In [6]:
# NCBI = ColumnCorpus(NCBI_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)

# BioNLP13PC = ColumnCorpus(BioNLP13PC_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)

# BC5CDR = ColumnCorpus(BC5CDR_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)


BC2GM = ColumnCorpus(BC2GM_data_folder, columns, 
                              train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)


# JNLPBA = ColumnCorpus(JNLPBA_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)

2019-09-06 14:02:52,563 Reading data from /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC2GM-IOB
2019-09-06 14:02:52,566 Train: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC2GM-IOB/train.tsv
2019-09-06 14:02:52,567 Dev: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC2GM-IOB/devel.tsv
2019-09-06 14:02:52,568 Test: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC2GM-IOB/test.tsv
2019-09-06 14:02:59,359 Reading data from /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/JNLPBA
2019-09-06 14:02:59,374 Train: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/JNLPBA/train.tsv
2019-09-06 14:02:59,375 Dev: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/JNLPBA/devel.tsv
2019-09-06 14:02:59,377 Test: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/JNLPBA/test.tsv


In [10]:
from flair.data import MultiCorpus
# multi_corpus = MultiCorpus([NCBI, BC5CDR])
multi_corpus = MultiCorpus([BC2GM])

In [8]:
#corpus: TaggedCorpus = read_data()
print(multi_corpus)

Corpus: 12574 train + 2519 dev + 5038 test sentences
Corpus: 18607 train + 1939 dev + 4260 test sentences


In [11]:
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = multi_corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

[b'<unk>', b'O', b'B-GENE', b'I-GENE', b'<START>', b'<STOP>']


In [12]:
# embedding_types: List[TokenEmbeddings] = [

#     WordEmbeddings('glove'),

# #     pretrained_word2vec,

#     # comment in this line to use character embeddings
#     # CharacterEmbeddings(),

#     # comment in these lines to use contextual string embeddings
#     FlairEmbeddings('news-forward'),
#     FlairEmbeddings('news-backward'),
# ]

# word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('pubmed-forward'), 
# FlairEmbeddings('pubmed-backward'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

embedding_types: List[TokenEmbeddings] = [CharacterEmbeddings(), PooledFlairEmbeddings('pubmed-backward', pooling='max'),
    PooledFlairEmbeddings('pubmed-forward', pooling='max')]

In [13]:
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

In [14]:
# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

In [15]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, multi_corpus)

In [ ]:
# 7. start training
# data_path = '/nfs/gns/literature/Santosh_Tirunagari/'
# if os.path.isdir(data_path+'/Models/') == False:
#     os.mkdir(data_path+'/Models/ner_flair_model')

In [16]:
trainer.train('../flair_models/ner/multi_bio_ner_model_gene',
              learning_rate=0.15,
              mini_batch_size=256,
              max_epochs=150)

2019-09-06 14:40:22,994 ----------------------------------------------------------------------------------------------------
2019-09-06 14:40:22,997 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
    (list_embedding_1): PooledFlairEmbeddings(
      (context_embeddings): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.5)
          (encoder): Embedding(275, 200)
          (rnn): LSTM(200, 1150, num_layers=3, dropout=0.5)
          (decoder): Linear(in_features=1150, out_features=275, bias=True)
        )
      )
    )
    (list_embedding_2): PooledFlairEmbeddings(
      (context_embeddings): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.5)
          (encoder): Embedding(275, 200)
          (rnn): LSTM(200, 1150, num_layers=3, dropout=0.5)
          (decoder): Linear(in_feature

2019-09-06 15:41:31,747 epoch 5 - iter 5/50 - loss 2.95175087 - samples/sec: 29.86
2019-09-06 15:42:14,342 epoch 5 - iter 10/50 - loss 2.85636824 - samples/sec: 31.14
2019-09-06 15:43:08,002 epoch 5 - iter 15/50 - loss 2.61749130 - samples/sec: 24.80
2019-09-06 15:44:17,073 epoch 5 - iter 20/50 - loss 2.49245648 - samples/sec: 20.51
2019-09-06 15:45:04,827 epoch 5 - iter 25/50 - loss 2.44870106 - samples/sec: 31.40
2019-09-06 15:46:02,966 epoch 5 - iter 30/50 - loss 2.43402804 - samples/sec: 23.23
2019-09-06 15:47:22,395 epoch 5 - iter 35/50 - loss 2.45374597 - samples/sec: 17.93
2019-09-06 15:48:45,252 epoch 5 - iter 40/50 - loss 2.50840209 - samples/sec: 17.13
2019-09-06 15:50:03,624 epoch 5 - iter 45/50 - loss 2.49597908 - samples/sec: 17.91
2019-09-06 15:50:58,838 ----------------------------------------------------------------------------------------------------
2019-09-06 15:50:58,840 EPOCH 5 done: loss 2.5451 - lr 0.1500
2019-09-06 15:52:59,895 DEV : loss 2.158275842666626 - sco

2019-09-06 17:11:07,953 epoch 11 - iter 15/50 - loss 1.89324166 - samples/sec: 16.60
2019-09-06 17:12:30,266 epoch 11 - iter 20/50 - loss 1.90248459 - samples/sec: 16.88
2019-09-06 17:13:51,483 epoch 11 - iter 25/50 - loss 1.85038838 - samples/sec: 17.27
2019-09-06 17:15:13,354 epoch 11 - iter 30/50 - loss 1.85949400 - samples/sec: 17.22
2019-09-06 17:16:35,219 epoch 11 - iter 35/50 - loss 1.82425220 - samples/sec: 16.77
2019-09-06 17:17:52,492 epoch 11 - iter 40/50 - loss 1.80554807 - samples/sec: 17.90
2019-09-06 17:19:16,145 epoch 11 - iter 45/50 - loss 1.78997192 - samples/sec: 16.81
2019-09-06 17:20:07,128 ----------------------------------------------------------------------------------------------------
2019-09-06 17:20:07,130 EPOCH 11 done: loss 1.7773 - lr 0.1500
2019-09-06 17:22:24,002 DEV : loss 1.3801870346069336 - score 0.7294
2019-09-06 17:22:47,069 BAD EPOCHS (no improvement): 0
2019-09-06 17:22:59,275 ---------------------------------------------------------------------

2019-09-06 18:15:29,517 epoch 17 - iter 25/50 - loss 1.64692995 - samples/sec: 30.97
2019-09-06 18:16:11,963 epoch 17 - iter 30/50 - loss 1.69573843 - samples/sec: 31.45
2019-09-06 18:16:54,460 epoch 17 - iter 35/50 - loss 1.65683244 - samples/sec: 30.92
2019-09-06 18:17:34,727 epoch 17 - iter 40/50 - loss 1.64681954 - samples/sec: 33.12
2019-09-06 18:18:19,463 epoch 17 - iter 45/50 - loss 1.62254354 - samples/sec: 29.64
2019-09-06 18:18:45,499 ----------------------------------------------------------------------------------------------------
2019-09-06 18:18:45,502 EPOCH 17 done: loss 1.5953 - lr 0.1500
2019-09-06 18:20:02,367 DEV : loss 1.147926688194275 - score 0.7701
2019-09-06 18:20:06,503 BAD EPOCHS (no improvement): 0
2019-09-06 18:20:12,797 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-06 18:20:23,882 epoch 18 - iter 0/50 - loss 1.30298185 - samples/se

2019-09-06 19:06:33,282 epoch 23 - iter 35/50 - loss 1.43426145 - samples/sec: 30.97
2019-09-06 19:07:13,165 epoch 23 - iter 40/50 - loss 1.41105938 - samples/sec: 33.46
2019-09-06 19:07:55,840 epoch 23 - iter 45/50 - loss 1.40279106 - samples/sec: 31.22
2019-09-06 19:08:23,662 ----------------------------------------------------------------------------------------------------
2019-09-06 19:08:23,663 EPOCH 23 done: loss 1.4075 - lr 0.1500
2019-09-06 19:09:34,805 DEV : loss 1.32192063331604 - score 0.7605
2019-09-06 19:09:38,218 BAD EPOCHS (no improvement): 2
2019-09-06 19:09:38,223 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-06 19:09:49,942 epoch 24 - iter 0/50 - loss 1.55168939 - samples/sec: 161.88
2019-09-06 19:10:31,962 epoch 24 - iter 5/50 - loss 1.57279338 - samples/sec: 31.49
2019-09-06 19:11:12,219 epoch 24 - iter 10/50 - loss 1.48669063 - samples/sec

...

2019-09-07 02:08:10,446 epoch 72 - iter 5/50 - loss 1.12505187 - samples/sec: 31.86
2019-09-07 02:08:51,483 epoch 72 - iter 10/50 - loss 1.09918520 - samples/sec: 32.58
2019-09-07 02:09:30,997 epoch 72 - iter 15/50 - loss 1.08235396 - samples/sec: 33.85
2019-09-07 02:10:12,718 epoch 72 - iter 20/50 - loss 1.06489149 - samples/sec: 31.52
2019-09-07 02:10:56,022 epoch 72 - iter 25/50 - loss 1.06666071 - samples/sec: 31.63
2019-09-07 02:11:36,422 epoch 72 - iter 30/50 - loss 1.06920014 - samples/sec: 33.21
2019-09-07 02:12:18,138 epoch 72 - iter 35/50 - loss 1.06208933 - samples/sec: 32.00
2019-09-07 02:12:57,214 epoch 72 - iter 40/50 - loss 1.07290378 - samples/sec: 34.02
2019-09-07 02:13:35,706 epoch 72 - iter 45/50 - loss 1.07195970 - samples/sec: 34.41
2019-09-07 02:14:01,779 ----------------------------------------------------------------------------------------------------
2019-09-07 02:14:01,781 EPOCH 72 done: loss 1.0633 - lr 0.0023
2019-09-07 02:15:16,725 DEV : loss 0.95612639188

2019-09-07 02:57:28,321 epoch 78 - iter 10/50 - loss 1.11522926 - samples/sec: 31.38
2019-09-07 02:58:06,827 epoch 78 - iter 15/50 - loss 1.08491416 - samples/sec: 34.72
2019-09-07 02:58:47,178 epoch 78 - iter 20/50 - loss 1.09607306 - samples/sec: 33.10
2019-09-07 02:59:27,566 epoch 78 - iter 25/50 - loss 1.09248735 - samples/sec: 33.02
2019-09-07 03:00:07,898 epoch 78 - iter 30/50 - loss 1.09943173 - samples/sec: 33.22
2019-09-07 03:00:47,930 epoch 78 - iter 35/50 - loss 1.07263423 - samples/sec: 33.09
2019-09-07 03:01:26,950 epoch 78 - iter 40/50 - loss 1.07376239 - samples/sec: 34.66
2019-09-07 03:02:05,377 epoch 78 - iter 45/50 - loss 1.07226283 - samples/sec: 35.03
2019-09-07 03:02:31,955 ----------------------------------------------------------------------------------------------------
2019-09-07 03:02:31,956 EPOCH 78 done: loss 1.0778 - lr 0.0012
2019-09-07 03:03:42,727 DEV : loss 0.9532095789909363 - score 0.8081
Epoch    77: reducing learning rate of group 0 to 5.8594e-04.
2

2019-09-07 03:44:50,092 epoch 84 - iter 10/50 - loss 1.13614296 - samples/sec: 32.80
2019-09-07 03:45:30,247 epoch 84 - iter 15/50 - loss 1.13492307 - samples/sec: 32.92
2019-09-07 03:46:11,255 epoch 84 - iter 20/50 - loss 1.11700444 - samples/sec: 32.39
2019-09-07 03:46:51,257 epoch 84 - iter 25/50 - loss 1.09826245 - samples/sec: 32.76
2019-09-07 03:47:31,487 epoch 84 - iter 30/50 - loss 1.08930714 - samples/sec: 32.83
2019-09-07 03:48:11,630 epoch 84 - iter 35/50 - loss 1.08731425 - samples/sec: 33.17
2019-09-07 03:48:52,505 epoch 84 - iter 40/50 - loss 1.10892669 - samples/sec: 32.62
2019-09-07 03:49:31,936 epoch 84 - iter 45/50 - loss 1.09630138 - samples/sec: 33.78
2019-09-07 03:49:56,871 ----------------------------------------------------------------------------------------------------
2019-09-07 03:49:56,873 EPOCH 84 done: loss 1.0986 - lr 0.0003
2019-09-07 03:51:06,927 DEV : loss 0.9529302716255188 - score 0.8078
2019-09-07 03:51:11,537 BAD EPOCHS (no improvement): 2
2019-09-

2019-09-07 04:32:44,460 epoch 90 - iter 15/50 - loss 1.08547913 - samples/sec: 34.64
2019-09-07 04:33:25,164 epoch 90 - iter 20/50 - loss 1.07175771 - samples/sec: 32.75
2019-09-07 04:34:07,705 epoch 90 - iter 25/50 - loss 1.07933050 - samples/sec: 31.64
2019-09-07 04:34:46,173 epoch 90 - iter 30/50 - loss 1.07972391 - samples/sec: 34.66
2019-09-07 04:35:27,152 epoch 90 - iter 35/50 - loss 1.07053935 - samples/sec: 32.60
2019-09-07 04:36:09,206 epoch 90 - iter 40/50 - loss 1.07860506 - samples/sec: 31.64
2019-09-07 04:36:48,109 epoch 90 - iter 45/50 - loss 1.07968124 - samples/sec: 34.53
2019-09-07 04:37:14,655 ----------------------------------------------------------------------------------------------------
2019-09-07 04:37:14,658 EPOCH 90 done: loss 1.0851 - lr 0.0001
2019-09-07 04:38:21,618 DEV : loss 0.9531370997428894 - score 0.8076
Epoch    89: reducing learning rate of group 0 to 7.3242e-05.
2019-09-07 04:38:25,975 BAD EPOCHS (no improvement): 4
2019-09-07 04:38:25,982 -------

{'test_score': 0.7911,
 'dev_score_history': [0.441,
  0.5697,
  0.5903,
  0.6402,
  0.6189,
  0.5898,
  0.7188,
  0.7066,
  0.6948,
  0.6896,
  0.7294,
  0.7376,
  0.7412,
  0.723,
  0.5703,
  0.746,
  0.7701,
  0.7311,
  0.723,
  0.7241,
  0.7726,
  0.7286,
  0.7605,
  0.7329,
  0.7056,
  0.7839,
  0.7659,
  0.7732,
  0.7884,
  0.7807,
  0.7764,
  0.7882,
  0.7748,
  0.7989,
  0.7972,
  0.7968,
  0.7801,
  0.8013,
  0.7939,
  0.7997,
  0.794,
  0.7951,
  0.8003,
  0.8014,
  0.8025,
  0.7935,
  0.8037,
  0.8049,
  0.8038,
  0.7958,
  0.8038,
  0.8046,
  0.8014,
  0.8042,
  0.8054,
  0.8061,
  0.807,
  0.8068,
  0.8068,
  0.8062,
  0.8055,
  0.806,
  0.8085,
  0.8069,
  0.8078,
  0.8076,
  0.8061,
  0.8082,
  0.8072,
  0.8086,
  0.8072,
  0.8071,
  0.8084,
  0.8078,
  0.8079,
  0.8074,
  0.8086,
  0.8081,
  0.8078,
  0.8075,
  0.8076,
  0.8072,
  0.8073,
  0.8078,
  0.8079,
  0.808,
  0.808,
  0.8076,
  0.8076,
  0.8076],
 'train_loss_history': [8.392884855270387,
  4.156689991950989,


In [32]:
from flair.models import TextClassifier
from flair.data import Sentence, Token

# load the model you trained
# gene_model = SequenceTagger.load('../flair_models/ner/multi_bio_ner_model_gene/best-model.pt')

# create example sentence
xx= 'For example, the chemical inhibition of IL4I1 activity may represent a new adjuvant strategy for the treatment of cancer by restoring specific anti-tumor immune responses.'

xx1 = 'These results suggested that during ischemic cardiomyopathy, initially neutrophil proteinase activates latent myocardial MMP which can degrade ECM, which continuously degrades if not controlled by TIMP, leading to ventricular dilatation and dysfunction.'

xx2= 'GR3027 antagonizes GABAA receptor potentiating neurosteroids and restores spatial learning and motor coordination in rats with chronic hyperammonemia and hepatic encephalopathy'

sentence = Sentence(xx2)

# predict tags and print
gene_model.predict(sentence)

print(sentence.to_tagged_string())

GR3027 antagonizes GABAA <B-GENE> receptor <I-GENE> potentiating neurosteroids and restores spatial learning and motor coordination in rats with chronic hyperammonemia and hepatic encephalopathy


In [31]:
from flair.models import TextClassifier
from flair.data import Sentence, Token

# load the model you trained
# disease_model = SequenceTagger.load('../flair_models/ner/multi_bio_ner_model_disease/best-model.pt')

# create example sentence
xx= 'For example, the chemical inhibition of IL4I1 activity may represent a new adjuvant strategy for the treatment of cancer by restoring specific anti-tumor immune responses.'

xx1 = 'These results suggested that during ischemic cardiomyopathy, initially neutrophil proteinase activates latent myocardial MMP which can degrade ECM, which continuously degrades if not controlled by TIMP, leading to ventricular dilatation and dysfunction.'

xx2= 'GR3027 antagonizes GABAA receptor potentiating neurosteroids and restores spatial learning and motor coordination in rats with chronic hyperammonemia and hepatic encephalopathy'

sentence = Sentence(xx2)

# predict tags and print
disease_model.predict(sentence)

print(sentence.to_tagged_string())

GR3027 antagonizes GABAA receptor potentiating neurosteroids and restores spatial learning and motor coordination in rats with chronic hyperammonemia <B-Disease> and hepatic <B-Disease> encephalopathy <I-Disease>
